In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
#set directory for importinh my_tools,py
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
from datetime import datetime
import glob
%matplotlib qt

Selecting the GEFS data and comparing them with ERA5 observations. Choose a variable of your liking

In [2]:
#set path. Change accoridngly
path_file = '/media/onno/Algemeen/Thesis/'
path = '/media/onno/Algemeen/Thesis/GFS_mean/compare_GFS/'

In [3]:
# # file_gfs = 'gefsrf2_env_wledit2000-10000_latavg_v300_control0-252h_6hourly_2x2_dec84-nov19.nc'
# # file_era = 'cfsr-cfsv2_env_wledit2000-10000_latavg_v300_79-19_6hourly_anom_from_smoothed04_clim_smoothed.nc'
# file_gfs = 'gefsrf2_phasevel_wledit2000-10000_latavg_v300_envgt15_control0-252h_6hourly_2x2_dec84-nov19_setvrange_-100to100.nc'
# file_era = 'cfsr-cfsv2_phasevel_wledit2000-10000_latavg_v300_envgt15_79-19_6hourly_anom_from_smoothed04_clim_setvrange_-100to100.nc'
# # file_gfs = 'gefsrf2_u300_control0-252h_6hourly_2x2_dec84-nov19.nc'
# # file_era = 'era5_u300_79-19_6hourly.nc'

#Looop over all variables to compute annual mean error
# variablez = ['envelope','phasespeed','u_wind']
variablez=['envelope','phasespeed']
for variable in variablez:
    print(variable)
    #get file names from imported dictionary
    file_gfs = file_dic[variable]['GFS']
    file_era = file_dic[variable]['ERA5']
    #open netcdf files as xarray datasets
    gfs = xr.open_dataset(path_file+file_gfs,decode_times=False).squeeze()
    era = xr.open_dataset(path_file+file_era,decode_times=False).squeeze()
    #change dimension names of ERA5 dataset
    if list(era.coords)[1]=='longitude':
        era=era.rename({'longitude':'lon'})
    if list(era.coords)[2]=='latitude':
        era=era.rename({'latitude':'lat'})

    #Convert dates to standard datetime values
    gfs['time']=my_tools.convert_date_gefs_r(gfs.time.values)
    era['time']=my_tools.convert_date_era_r(era.time.values)

    #Select only data for 24h intervals
    gfs = gfs.sel(lead=gfs.lead.values[[bool(i) for i in gfs.lead%24==0]])

    #Calculating daily mean errors 


    t1 = datetime.now()
    for i in range(11):
        #Select days for which GEFS data is present and make datasets equally long
        gfs_d = gfs.sel(time=[i for i in pd.date_range('1984-12-1','2019-11-{}'.format(30-i))],lead=i*24)
        era_d = gfs.sel(time=[i for i in pd.date_range('1984-12-{}'.format(1+i),'2019-11-30')],lead=0) 
        print(i)
        #Make boolean array for each day that is present in season        
        #Check if relevant dates are correct
        assert(gfs_d.time.values[0]==(era_d.time.values[0]-pd.Timedelta(i,'d')))
        #for each dataset calculate mean 
        gfs_season_mean = gfs_d.mean(dim='time')
        era_season_mean = era_d.mean(dim='time')
        #Subtract reforecast dataset with reanalysis dataset to get mean error
        diff = gfs_season_mean - era_season_mean
        #save mean errors for each lead day to new netcdf file
        diff.to_netcdf(path +'GFS_mean_error_{}_day_{:02d}_globe_mars.nc'.format(variable,i))
        
    #Measure duration of calculation

    t2 = datetime.now()
    print(t2-t1)

envelope


FileNotFoundError: [Errno 2] No such file or directory: b'/media/onno/Algemeen/Thesis/gefsrf2_env_wledit2000-10000_latavg_v300_control0-252h_6hourly_2x2_dec84-nov19.nc'

In [6]:
t1 = datetime.now()
for i in range(1,11):
    forecast = gfs_nh.sel(time=[my_tools.convert_date_gefs(i) for i in pd.date_range('2014-12-1','2019-11-{}'.format(30-i))],lead=i*24)
    reanalysis = era_nh.sel(time=[my_tools.convert_date_era(i) for i in pd.date_range('2014-12-{}'.format(1+i),'2019-11-30')])
    assert(my_tools.convert_date_gefs_r(float(forecast.time[0]+forecast.lead.values))==my_tools.convert_date_era_r(float(reanalysis.time[0])))
    reanalysis['time'] = forecast.time.values
#     forecast_mean = forecast.mean(dim='time')
#     reanalysis_mean = reanalysis.mean(dim='time')
    diff2 = forecast - reanalysis
    diff2 = diff2.mean(dim='time')
    break
    diff.to_netcdf('/media/onno/Algemeen/Thesis/GFS_mean/bias_GFS_forecast_phasespeed_day{:02d}.nc'.format(i))
    print(i)
t2 = datetime.now()
print(t2-t1)


0:00:01.863019
